In [1]:
import sys
sys.path.append("../dwaveutils/dwavetools")
sys.path.append("../dwaveutils/probrep")
from dictrep import DictRep
from dwavetools import dense_connect_2000Q
from dwave.system.samplers import DWaveSampler
import networkx as nx
import dwave_networkx as dnx
from collections import defaultdict

### First, get the chip data

In [2]:
sampler = DWaveSampler()
wqubits = sampler.properties['qubits']
wcouplers = sampler.properties['couplers']
chipdata = {'wqubits': wqubits, 'wcouplers': wcouplers}

### Create large logical qubits from dense, rectangular chains and connect them
The way this works is as follows: the dense_connect_2000Q function creates a dense chain connection from some starting unit cell to a final unit cell in a rectangular box with R rows and C columns of unit cells. In particular, it starts the chain with qubit qi, then moves from left to right (up to down) by C (by R) unit cells while chaining nearest neighbor unit cells within the R X C rectangle.

In [6]:
# create the first logical qubit Hamiltonian by chaining 8 x 10 unit cells in upper-left corner
logq1 = dense_connect_2000Q(chipdata, qi = 0, R = 10, C = 8, hval =  'h', Jval = 'J')

Going to try and connect unit cell 0 with unit cell 151
Successfully created a dense chain with 639 qubits.


In [7]:
# create the second logical qubit by chaining 8 x 10 unit cells in top right corner
logq2 = dense_connect_2000Q(chipdata, qi = 64, R = 10, C = 8, hval = 'h', Jval = 'J')

Going to try and connect unit cell 8 with unit cell 159
Successfully created a dense chain with 635 qubits.


In [8]:
# create the third logical qubit by chaining 5 x 16 unit cells at bottom
logq3 = dense_connect_2000Q(chipdata, qi = 1280, R = 5, C = 16, hval = 'h', Jval = 'J')

Going to try and connect unit cell 160 with unit cell 239
Successfully created a dense chain with 637 qubits.


In [10]:
# Now, we connect the logical rectangular qubits with couplings between them
inter_uc_couplings = {(60, 68): 'J2', (1288, 1160): 'J1', (1392, 1264): 'J1'}

In [18]:
# combine them all together
dicts = [logq1, logq2, logq3, inter_uc_couplings]
super_dict = {}
for d in dicts:
    for k, v in d.items():
        super_dict[k].add(v)

KeyError: (0, 0)

In [19]:
super_dict

{}

In [15]:
# get working qubits and couplers from the solver
sampler = DWaveSampler()
wqubits = sampler.properties['qubits']
wcouplers = sampler.properties['couplers']
print(sampler.properties['chip_id'])

DW_2000Q_2


In [12]:
# create a list of all qubits and find the set difference between working and all
allqubits = [q for q in range(2048)]
brokenqubits = sorted(list(set(allqubits) - set(wqubits)))
print(brokenqubits)

[577, 691, 1012, 1105, 1276, 1730, 1776, 1858]


In [37]:
# create a list of all couplers and find the set difference between working and all
allcouplers = []
for q in range(2048):
    if q % 8 < 4:
        allcouplers.extend([[q, (q-q%8)+4], [q, (q-q%8)+5], [q, (q-q%8)+6], [q, (q-q%8)+7]])
        if q + 128 < 2047:
            allcouplers.append([q, q+128])
    else:
        if ((q-q%8+4 + 4) % 128) == 0:
            continue
        else:
            allcouplers.append([q, q+8])
brokencouplers = [coupler for coupler in allcouplers if coupler not in wcouplers]
print(brokencouplers)

[[449, 577], [563, 691], [577, 580], [577, 581], [577, 582], [577, 583], [577, 705], [691, 692], [691, 693], [691, 694], [691, 695], [691, 819], [715, 718], [977, 1105], [1004, 1012], [1008, 1012], [1009, 1012], [1010, 1012], [1011, 1012], [1012, 1020], [1105, 1108], [1105, 1109], [1105, 1110], [1105, 1111], [1105, 1233], [1235, 1238], [1235, 1239], [1268, 1276], [1272, 1276], [1273, 1276], [1274, 1276], [1275, 1276], [1602, 1730], [1648, 1776], [1730, 1732], [1730, 1733], [1730, 1734], [1730, 1735], [1730, 1858], [1776, 1780], [1776, 1781], [1776, 1782], [1776, 1783], [1776, 1904], [1858, 1860], [1858, 1861], [1858, 1862], [1858, 1863], [1858, 1986]]
